In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook, get_unique_name
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
from hydradxapi import HydraDX
from pprint import pprint
import requests

import json, os


In [2]:
with open('config/arbconfig.txt', 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

order_books = {}
for arb_cfg in arb_list:
    tkn_pair = arb_cfg['order_book']
    if tkn_pair not in order_books:
        resp = requests.get('https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1])
        y = resp.json()
        order_books[tkn_pair] = y['result'][tkn_pair[0] + tkn_pair[1]]
    


In [3]:
RPC="wss://hydradx-rpc.dwellir.com"
N = 24
metadata = []
with HydraDX(RPC) as chain:
    for i in range(N):
        try:
            md = chain.api.registry.asset_metadata(i)
            metadata.append({'symbol': md.symbol, 'id': md.asset_id, 'decimals': md.decimals},)
        except:
            continue

    asset_list = []
    fees = {}
    tokens = {}
    asset_map = {}
    
    for i in range(len(metadata)):
        try:
            state = chain.api.omnipool.asset_state(metadata[i]['id'])
        except:
            continue
            
        fee = chain.api.fees.asset_fees(metadata[i]['id'])
    
        tkn = get_unique_name(asset_list, metadata[i]['symbol'])
        asset_list.append(tkn)
        asset_map[i] = tkn
        tokens[tkn] = {
            'liquidity': state.reserve / 10**metadata[i]['decimals'],
            'LRNA': state.hub_reserve / 10**12
        }
        fees[tkn] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
            

pprint(tokens)


{'ASTR': {'LRNA': 9082.738326360857, 'liquidity': 4613307.689932842},
 'BNC': {'LRNA': 6960.530069554551, 'liquidity': 723868.8934341213},
 'CFG': {'LRNA': 8851.608850641, 'liquidity': 602207.3975931228},
 'DAI': {'LRNA': 13110.676857870463, 'liquidity': 332743.9021445043},
 'DAI001': {'LRNA': 31824.663044315806, 'liquidity': 799910.0687595578},
 'DOT': {'LRNA': 70109.9798847654, 'liquidity': 429253.0549350427},
 'GLMR': {'LRNA': 5249.607197601644, 'liquidity': 677608.617803954},
 'HDX': {'LRNA': 19546.514674768685, 'liquidity': 117711721.7149292},
 'USDT': {'LRNA': 85405.85683167279, 'liquidity': 2150035.105705},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41825.898662570326, 'liquidity': 31.07199565},
 'WETH': {'LRNA': 21611.81073019557, 'liquidity': 308.36181823539135},
 'WETH001': {'LRNA': 71425.47037215637, 'liquidity': 1019.4586640314313},
 'ZTG': {'LRNA': 4538.957735895216, 'liquidity': 3133789.308258377},
 'iBTC': {'LRNA': 34924.27081821849

In [4]:

ob_objs = {}
for tkn_pair in order_books:
    ob = parse_kraken_orderbook(order_books[tkn_pair])
    ob_obj = OrderBook(
        bids=[[bid['price'], bid['amount']] for bid in ob['bids']],
        asks=[[ask['price'], ask['amount']] for ask in ob['asks']]
    )
    ob_objs[tkn_pair] = ob_obj
    
order_book_asset_list = []
for tkn_pair in order_books:
    for tkn in tkn_pair:
        if tkn not in order_book_asset_list:
            order_book_asset_list.append(tkn)

cex_fee = 0.0016
buffer = 0.0002

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0005
# asset_fee = 0.0025


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [5]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('DOT', 'iBTC'): ('DOT', 'XBT'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'DAI'): ('XETH', 'ZUSD'),
 ('WETH', 'DAI001'): ('XETH', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUS

In [6]:
all_swaps = get_arb_swaps(op_state, cex, order_book_map, buffer=buffer)
for tkn_pair in all_swaps:
    if all_swaps[tkn_pair] != []:
        print(tkn_pair)
        print(all_swaps[tkn_pair])

('DOT', 'DAI001')
[{'dex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'DAI001', 'price': 4.1282, 'amount': 50.54401397705078, 'max_sell': 208.2223218772769}, 'cex': {'trade': 'sell', 'buy_asset': 'USD', 'sell_asset': 'DOT', 'price': 4.1277871799999994, 'amount': 50.54401397705078}}]
('WETH', 'USDT')
[{'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'USDT', 'price': 1779.66, 'amount': 0.4696312236785889, 'max_sell': 832.769665953803}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XETH', 'price': 1779.482034, 'amount': 0.4696312236785889}}]
('WETH', 'DAI001')
[{'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'DAI001', 'price': 1779.66, 'amount': 0.166425256729126, 'max_sell': 503.63024120613335}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XETH', 'price': 1779.482034, 'amount': 0.166425256729126}}]
('WETH001', 'USDT')
[{'dex': {'trade': 'buy', 'buy_asset': 'WETH001', 'sell_asset': 'USDT', 'price': 1779.66, 'amount': 0.93833224

In [7]:
state = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit = calculate_profit(init_agent, agent, asset_map)

print(profit)
            

{'HDX': 0, 'USD': 8.142181992530823, 'BTC': 8.487701416015625e-05, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0.0, 'vDOT': 0, 'GLMR': 0}
